In [1]:
import pandas as pd
import psycopg2

In [2]:
def create_db():
    conn = psycopg2.connect("host=localhost dbname=postgres user=postgres password=root")
    conn.set_session(autocommit=True)
    cur = conn.cursor()

    cur.execute("DROP DATABASE IF EXISTS accounts")
    cur.execute("CREATE DATABASE accounts")

    cur.close()
    conn.close()
    

    conn = psycopg2.connect("host=localhost dbname=accounts user=postgres password=root")
    cur = conn.cursor()

    return cur, conn
    

In [3]:
def drop_tables(cur, conn):
    for query in create_table_queries:
        cur.execute(query)
        conn.commit()

In [4]:
Accountscountry = pd.read_csv("Wealth-AccountsCountry.csv")

In [5]:
Accountscountry.head()


,Code,Long Name,Income Group,Region,Lending category,Other groups,Currency Unit,Latest population census,Latest household survey,Special Notes,...,Source of most recent Income and expenditure data,Vital registration complete,Latest agricultural census,Latest industrial data,Latest trade data,Latest water withdrawal data,2-alpha code,WB-2 code,Table Name,Short Name
0,ALB,Republic of Albania,Upper middle income,Europe & Central Asia,IBRD,NaN,Albanian lek,2020 (expected),"Demographic and Health Survey, 2017/18",NaN,...,Living Standards Measurement Study Survey (LSM...,Yes,2012,2013.0,2018.0,2006.0,AL,AL,Albania,Albania
1,ARG,Argentine Republic,Upper middle income,Latin America & Caribbean,IBRD,NaN,Argentine peso,2020 (expected),"Multiple Indicator Cluster Survey, 2019/20",NaN,...,"Integrated household survey (IHS), 2016",Yes,2008,2002.0,2018.0,2011.0,AR,AR,Argentina,Argentina
2,ARM,Republic of Armenia,Upper middle income,Europe & Central Asia,IBRD,NaN,Armenian dram,2020 (expected),"Demographic and Health Survey, 2015/16",NaN,...,"Integrated household survey (IHS), 2016",Yes,2014,NaN,2018.0,2012.0,AM,AM,Armenia,Armenia
3,AUS,Commonwealth of Australia,High income,East Asia & Pacific,NaN,NaN,Australian dollar,2016,NaN,Fiscal year end: June 30; reporting period for...,...,"Expenditure survey/budget survey (ES/BS), 2010",Yes,2015-2016,2013.0,2018.0,2013.0,AU,AU,Australia,Australia
4,AUT,Republic of Austria,High income,Europe & Central Asia,NaN,Euro area,Euro,2011. Population figures compiled from adminis...,NaN,A simple multiplier is used to convert the nat...,...,"Income survey (IS), 2015",Yes,2010,2014.0,2018.0,2010.0,AT,AT,Austria,Austria


In [6]:
Accountscountry_clean = Accountscountry[['Code', 'Short Name', 'Table Name', 'Long Name', 'Currency Unit']]

In [7]:
Accountscountry_clean.head()
Accountscountry_clean.drop_duplicates(subset=['Code'], keep='first', inplace=True)
Accountscountry_clean.count()

C:\Users\vishv\AppData\Local\Temp\ipykernel_9656\3772070207.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Accountscountry_clean.drop_duplicates(subset=['Code'], keep='first', inplace=True)


Code             146
Short Name       146
Table Name       146
Long Name        146
Currency Unit    146
dtype: int64

In [8]:
Accountsdata = pd.read_csv("Wealth-AccountData.csv")

In [9]:
Accountsdata.head()


,Country Name,Country Code,Series Name,Series Code,1995 [YR1995],1996 [YR1996],1997 [YR1997],1998 [YR1998],1999 [YR1999],2000 [YR2000],...,2009 [YR2009],2010 [YR2010],2011 [YR2011],2012 [YR2012],2013 [YR2013],2014 [YR2014],2015 [YR2015],2016 [YR2016],2017 [YR2017],2018 [YR2018]
0,Albania,ALB,Human capital (constant 2018 US$),NW.HCA.TO,44900000000.00,43400000000.00,37100000000.00,38800000000.00,42200000000.00,43600000000.00,...,6.610000e+10,6.810000e+10,6.850000e+10,7.080000e+10,7.160000e+10,7.250000e+10,7.370000e+10,75700000000.00,78000000000.00,81200000000.00
1,Albania,ALB,Human capital per capita (constant 2018 US$),NW.HCA.PC,14072.79,13708.99,11794.10,12399.48,13573.26,14118.13,...,2.258200e+04,2.336286e+04,2.359058e+04,2.439397e+04,2.473162e+04,2.509002e+04,2.559371e+04,26326.26,27160.86,28333.68
2,Albania,ALB,"Human capital per capita, employed female (con...",NW.HCA.FEMP.PC,3468.86,3355.55,2869.00,3013.39,3259.89,3424.01,...,2.825340e+03,2.889150e+03,2.848520e+03,2.930230e+03,2.954270e+03,2.957420e+03,2.987550e+03,3072.50,3178.42,3312.93
3,Albania,ALB,"Human capital per capita, employed male (const...",NW.HCA.MEMP.PC,6507.93,6353.74,5466.69,5901.07,6492.73,6872.02,...,1.525319e+04,1.578992e+04,1.579854e+04,1.647360e+04,1.681013e+04,1.699138e+04,1.730701e+04,17817.10,18445.94,19245.19
4,Albania,ALB,"Human capital per capita, female (constant 201...",NW.HCA.FEMA.PC,4893.01,4737.86,4059.32,4191.45,4536.97,4695.07,...,3.529140e+03,3.613610e+03,3.603690e+03,3.683810e+03,3.696740e+03,3.719610e+03,3.767630e+03,3872.14,3992.19,4161.14


In [10]:
Accountsdata.columns

Index(['Country Name', 'Country Code', 'Series Name', 'Series Code',
       '1995 [YR1995]', '1996 [YR1996]', '1997 [YR1997]', '1998 [YR1998]',
       '1999 [YR1999]', '2000 [YR2000]', '2001 [YR2001]', '2002 [YR2002]',
       '2003 [YR2003]', '2004 [YR2004]', '2005 [YR2005]', '2006 [YR2006]',
       '2007 [YR2007]', '2008 [YR2008]', '2009 [YR2009]', '2010 [YR2010]',
       '2011 [YR2011]', '2012 [YR2012]', '2013 [YR2013]', '2014 [YR2014]',
       '2015 [YR2015]', '2016 [YR2016]', '2017 [YR2017]', '2018 [YR2018]'],
      dtype='object')

In [11]:
columns_to_drop = ['1996 [YR1996]', '1997 [YR1997]', '1998 [YR1998]', '1999 [YR1999]', '2001 [YR2001]', '2002 [YR2002]', '2003 [YR2003]', '2004 [YR2004]', '2006 [YR2006]', '2007 [YR2007]', '2008 [YR2008]', '2009 [YR2009]', '2011 [YR2011]', '2012 [YR2012]', '2013 [YR2013]', '2014 [YR2014]', '2016 [YR2016]', '2017 [YR2017]']


In [12]:
Accountsdata.drop(columns=columns_to_drop, inplace = True, axis=1)


In [15]:

Accountsdata.head()
Accountsdata.dtypes



Country Name      object
Country Code      object
Series Name       object
Series Code       object
1995 [YR1995]     object
2000 [YR2000]     object
2005 [YR2005]    float64
2010 [YR2010]    float64
2015 [YR2015]    float64
2018 [YR2018]     object
dtype: object

In [16]:
Accountsseries = pd.read_csv("Wealth-AccountSeries.csv")
Accountsseries.head()
Accountsseries.dtypes

Code                                   object
Indicator Name                         object
Long definition                        object
Source                                 object
Topic                                  object
Unit of measure                        object
Periodicity                            object
Reference period                       object
Statistical concept and methodology    object
Previous Indicator Code                object
Previous Indicator Name                object
dtype: object

In [17]:
Accountsseries = Accountsseries[['Code', 'Topic', 'Indicator Name', 'Long definition']]

In [18]:
Accountsseries.head()

,Code,Topic,Indicator Name,Long definition
0,NW.HCA.TO,Human capital,Human capital (constant 2018 US$),Human capital is computed as the present value...
1,NW.HCA.PC,Human capital,Human capital per capita (constant 2018 US$),Human capital is computed as the present value...
2,NW.HCA.FEMP.PC,Human capital,"Human capital per capita, employed female (con...",Human capital is computed as the present value...
3,NW.HCA.MEMP.PC,Human capital,"Human capital per capita, employed male (const...",Human capital is computed as the present value...
4,NW.HCA.FEMA.PC,Human capital,"Human capital per capita, female (constant 201...",Human capital is computed as the present value...


In [19]:
cur, conn = create_db()

In [20]:
accountscountry_table_create = ("""CREATE TABLE IF NOT EXISTS accountscountry(country_code varchar PRIMARY KEY, short_name varchar, table_name varchar, long_name varchar, currency_unit varchar)""")

In [21]:
cur.execute(accountscountry_table_create)
conn.commit()

In [22]:
accountsdata_table_create = ("""CREATE TABLE IF NOT EXISTS accountsdata(country_name varchar, country_code varchar, series_name varchar, series_code varchar, year_1995 varchar, year_2000 varchar, year_2005 float, year_2010 float, year_2015 float, year_2018 varchar)""")
cur.execute(accountsdata_table_create)
conn.commit()

In [23]:
accountsseries_table_create = ("""CREATE TABLE IF NOT EXISTS accountsseries(series_code varchar, topic varchar, indicator_name varchar, long_definition varchar)""")
cur.execute(accountsseries_table_create)
conn.commit()

In [24]:
accountscountry_table_insert = ("""INSERT INTO accountscountry(country_code, short_name, table_name, long_name, currency_unit) VALUES (%s, %s, %s, %s, %s)""")

In [25]:
for i, row in Accountscountry_clean.iterrows():
    cur.execute(accountscountry_table_insert, list(row))

In [26]:
conn.commit()

In [27]:
accountsdata_table_insert = ("""INSERT INTO accountsdata(country_name, country_code, series_name, series_code, year_1995, year_2000, year_2005, year_2010, year_2015, year_2018) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)""")

In [28]:
for i, row in Accountsdata.iterrows():
    cur.execute(accountsdata_table_insert, list(row))

In [29]:
conn.commit()


In [30]:
accountsseries_table_insert = ("""INSERT INTO accountsseries(series_code, topic, indicator_name, long_definition) VALUES (%s, %s, %s, %s)""")

In [31]:
for i, row in Accountsseries.iterrows():
    cur.execute(accountsseries_table_insert, list(row))

In [32]:
conn.commit()

In [35]:
cur.execute("SELECT * FROM accountscountry LIMIT 5;")
rows = cur.fetchall()
columns = [desc[0] for desc in cur.description]
accountcountrys_data = pd.DataFrame(rows, columns=columns)
print(accountcountrys_data)

  country_code short_name table_name                  long_name  \
0          ALB    Albania    Albania        Republic of Albania   
1          ARG  Argentina  Argentina         Argentine Republic   
2          ARM    Armenia    Armenia        Republic of Armenia   
3          AUS  Australia  Australia  Commonwealth of Australia   
4          AUT    Austria    Austria        Republic of Austria   

       currency_unit  
0       Albanian lek  
1     Argentine peso  
2      Armenian dram  
3  Australian dollar  
4               Euro  


In [36]:
cur.execute("SELECT * FROM accountsdata LIMIT 5;")
rows = cur.fetchall()
columns = [desc[0] for desc in cur.description]
accountdata_data = pd.DataFrame(rows, columns=columns)
print(accountdata_data)

  country_name country_code  \
0      Albania          ALB   
1      Albania          ALB   
2      Albania          ALB   
3      Albania          ALB   
4      Albania          ALB   

                                         series_name     series_code  \
0                  Human capital (constant 2018 US$)       NW.HCA.TO   
1       Human capital per capita (constant 2018 US$)       NW.HCA.PC   
2  Human capital per capita, employed female (con...  NW.HCA.FEMP.PC   
3  Human capital per capita, employed male (const...  NW.HCA.MEMP.PC   
4  Human capital per capita, female (constant 201...  NW.HCA.FEMA.PC   

        year_1995       year_2000     year_2005     year_2010     year_2015  \
0  44900000000.00  43600000000.00  5.440000e+10  6.810000e+10  7.370000e+10   
1        14072.79        14118.13  1.807230e+04  2.336286e+04  2.559371e+04   
2         3468.86         3424.01  2.222120e+03  2.889150e+03  2.987550e+03   
3         6507.93         6872.02  1.168935e+04  1.578992e+04  1

In [38]:
cur.execute("SELECT * FROM accountsseries LIMIT 5;")
rows = cur.fetchall()
columns = [desc[0] for desc in cur.description]
accountseries_data = pd.DataFrame(rows, columns=columns)
print(accountseries_data)

      series_code          topic  \
0       NW.HCA.TO  Human capital   
1       NW.HCA.PC  Human capital   
2  NW.HCA.FEMP.PC  Human capital   
3  NW.HCA.MEMP.PC  Human capital   
4  NW.HCA.FEMA.PC  Human capital   

                                      indicator_name  \
0                  Human capital (constant 2018 US$)   
1       Human capital per capita (constant 2018 US$)   
2  Human capital per capita, employed female (con...   
3  Human capital per capita, employed male (const...   
4  Human capital per capita, female (constant 201...   

                                     long_definition  
0  Human capital is computed as the present value...  
1  Human capital is computed as the present value...  
2  Human capital is computed as the present value...  
3  Human capital is computed as the present value...  
4  Human capital is computed as the present value...  
